In [7]:
#Kiran Rao

In [8]:
#importing pandas
import pandas as pd

In [17]:
#creating df1, df2 and df_empty dataframes which can be globally accessible
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df_empty = pd.DataFrame()

#function process_articles takes one argument: filename
def process_articles(filename):
    ##2 use read_csv() to read file
    global df1
    df1 = pd.read_csv(filename, delimiter = ',', header = 0, encoding_errors = 'ignore')

    
    ##3 use pandas for processing
    #1 assign numerical identifiers and store it in a new column ID
    #a new cloumn 'ID' is created maintaining the order of the rows by using the function reset_index()
    df1['ID'] = df1.reset_index().index
    
    
    #2 convert all numeric values to integer
    #All publication Year values are converted to int if numeric and -9999 if non-numeric
    #Here it first checks if the value is null, if true, 0 is assigned
    #Then it checks if the value is an interger, if true it keeps it as it is
    #it then checks if the value contains a decimal point, if true, the value before the decimal point is kept
    #it checks if the string value is numeric, if true it converts to int
    #else if the value is non numeric, it assigns -9999
    df1['Publication Year'] = df1['Publication Year'].apply(lambda x: 0 if pd.isnull(x) else (x if isinstance(x, int) == True else(int(x.split('.')[0]) if '.' in x else (int(x) if x.isnumeric() else -9999))))
    
    #3 replace no author by an empty string
    #Here it first extracts the 'Authors' that has 'no authors' value into a series s
    #replaces such fields by an empty string
    #counts the number of such fields and stores it in c_auth
    s = df1[df1['Authors'] == 'no author']['Authors']
    df1.loc[s.index, 'Authors'] = ""
    c_auth = len(s)

    #4 represent missing entry in abstract by an empty string
    #counts the number of fields with an empty string for 'Abstract' and stores it in c_abst
    #replaces such fields with an empty string
    c_abst = df1['Abstract'].isna().sum()
    df1.loc[df1['Abstract'].isnull(), 'Abstract'] = ""
    
    #5 count the number of empty DOIs
    #counts the number of fields with an empty 'DOI' and stores it in c_doi
    #also checks if there are any empty DOIs, if present, it proceeds to match the DOIs
    c_doi = df1['DOI'].isna().sum()
    c_match_doi = 0
    if c_doi>0:
        #5a replace empty DOIs by an empty string
        #replaces empty fields with an empty string
        df1.loc[df1['DOI'].isnull(), 'DOI'] = ""
        
        #checks if there are any Unnamed columns. If present, it drops them
        df1 = df1.loc[:,~df1.columns.str.contains('^Unnamed')]

        #5b partition the dataframes: df_empty and df2
        #creates two dataframes df_empty with all the rows that do not have a DOI and df2 with the rows that contains a DOI
        global df_empty
        global df2
        df_empty = df1[df1['DOI'] == ""]
        df2 = df1[df1['DOI']!= ""]
        
        #checks if the length of df2 and df_empty combined matches the length of df1
        assert len(df1)==len(df_empty)+len(df2), 'DF PARTITION ERROR'
        
        #resets the indexes of df_empty and df2 to continuous index
        df_empty = df_empty.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)
        
        #5c DOI match
        #to match the DOI 2 new dataframes df_match_empty and df_match_d2 are created
        #Document Title and DOI values are extracted from the dataframes df_empty and df2 respectively into df_match_empty and df_match_df2
        df_match_empty = df_empty[['Document Title', 'DOI']]
        df_match_df2 = df2[['Document Title', 'DOI']]
        
        #DOI is matched from df_match_empty and df_match_df2 using the merge function and is stores in m
        #the column with the matched DOI is renamed as Possible DOI
        m = pd.merge(df_match_empty, df_match_df2, how = 'left', on = 'Document Title')
        m = m.rename(columns = {'DOI_y': 'Possible DOI'})
        
        #number of matched DOIs are counted and is stored in c_match_doi
        c_match_doi = len(m.dropna())
        
        #the Possible DOI column from m is concatenated with columns in df_empty
        df_empty = pd.concat([df_empty, m['Possible DOI']], axis = 1)

        #Here df2 dataframe is merged with df_empty to combine everything back to one single dataframe
        #To bring it back to the original form, the values are sorted using the column ID
        #To make the Possible DOI column uniform, all the empty values are replaced by an empty string 
        final_df = df2.merge(df_empty, how = 'outer')
        final_df = final_df.sort_values(by = 'ID')
        final_df.loc[final_df['Possible DOI'].isnull(), 'Possible DOI'] = ""
        
        #uncomment this to view the final result
#         print(final_df)

    #did not replace the rest of the columns with empty values by an empty string as it was not asked
        
    #6 print message
    print("Number missing: Authors = ", c_auth, "; Abstracts = ", c_abst,"; DOIs = ", c_doi, "; Number of DOIs matched = ", c_match_doi)
    
    #was not sure if we had to return the values as it was not mentioned
#     return df1, df_empty, df2

In [18]:
##1 execute the function process_articles(filename), filename being a csv file
process_articles("research1.csv")

Number missing: Authors =  10 ; Abstracts =  8 ; DOIs =  16 ; Number of DOIs matched =  2
